# **Laboratorio pyspark(Bow)**

> **AUTOR :** Sarco Jacinto Daniel Eduardo.

> **CODIGO :** 174452.

##**Librerias Utilizar**##

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 67.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=70885cf31d89f721a0884fee0ab361581d86cdf7a17ea76e3a1fd5d2d9442952
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
sc = SparkContext()

In [77]:
import math
from pyspark.sql.functions import *

>**Bag of Word**

In [103]:
def Preprocesamiento(pNombreArchivo):
    #Arreglo donde almacenaremos nuestras tuplas
    ArregloTuplas=[]
    #Leemos el archivo .txt
    novaRDD = sc.textFile(pNombreArchivo)
    #Eliminarmos las lineas vacias del archivo  
    novaRDD = novaRDD.filter(lambda x : x is not '').collect()
    i=1#Contador para defenir el IdDocumento que estamos trabajando
    for Linea in novaRDD:#Cada linea del archivo .txt representa un documento
      Tupla=(i,Linea)#Tiene la forma (idDocumento,Una linea del documento)
      ArregloTuplas.append(Tupla)#Agregamos la tupla a nuestro arreglo
      i+=1#Incrementamos el contador
    return ArregloTuplas#Retornamos el arreglo de tuplas(data)
def Get_bow(pCorpus,pParticiones=4):
  linesRDD=sc.parallelize(pCorpus,pParticiones)
  parTokenRDD=linesRDD.flatMap(lambda x: [((x[0],i.lower()),1) for i in x[1].split()])
  return parTokenRDD

In [104]:
Data=Preprocesamiento("twitts.txt")
BoW=Get_bow(Data)
BoW.take(20)

[((1, 'protester'), 1),
 ((1, 'stage'), 1),
 ((1, 'diein'), 1),
 ((1, 'protest'), 1),
 ((1, 'store'), 1),
 ((1, 'nyc'), 1),
 ((1, 'anger'), 1),
 ((1, 'misplace'), 1),
 ((1, 'retweet'), 1),
 ((1, 'agree'), 1),
 ((2, 'apple'), 1),
 ((2, 'intraday'), 1),
 ((2, 'comment'), 1),
 ((2, 'update'), 1),
 ((2, 'range'), 1),
 ((2, 'premium'), 1),
 ((2, 'user'), 1),
 ((2, 'aapl'), 1),
 ((2, 'stockaction'), 1),
 ((2, 'trading'), 1)]

>TF

In [105]:
def Get_frecuencias(RDD):
  #obtenemos la frecuencia de palabras por documento
  return RDD.reduceByKey(lambda x,y:x+y)

In [116]:
def Get_TF(RDD):
  frecuenciasRDD=Get_frecuencias(RDD)
  # Mapear el RDD
  return frecuenciasRDD.map(lambda x: (x[0][1],(x[0][0],x[1])))

In [117]:
NroDoc=len(Data)
BoW=Get_bow(Data)
TF=Get_TF(BoW)
TF.take(10)

[('protester', (1, 1)),
 ('diein', (1, 1)),
 ('aapl', (2, 1)),
 ('stockaction', (2, 1)),
 ('iphone', (3, 1)),
 ('shit', (4, 1)),
 ('ridiculous', (5, 1)),
 ('suicide', (8, 1)),
 ('turkey', (8, 1)),
 ('ipod', (9, 1))]

>IDF

In [107]:
def Get_IDF(RDD,NroDoc):
  FRDD=Get_frecuencias(RDD)
  # Mapeamos para colocar cada token como valor clave.
  TRDD=FRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
  # Extraemos el token y su ocurrencia por documento
  TODRDD=TRDD.map(lambda x:(x[0],x[1][2]))
  # contamos la contidad de documentos que contienen un token
  TRDD=TODRDD.reduceByKey(lambda x,y:x+y)
  #calculamos el logaritmo para el IDF
  IDFRDD=TRDD.map(lambda x: (x[0],math.log10(NroDoc/x[1])))
  return IDFRDD

In [112]:
NroDoc=len(Data)
BoW=Get_bow(Data)
IDF=Get_IDF(BoW,NroDoc)
IDF.take(10)

[('aapl', 0.5944950710074257),
 ('stockaction', 2.0249643045063834),
 ('feedback', 3.104145550554008),
 ('iwatch', 2.627024295834346),
 ('accessory', 2.979206813945708),
 ('turn', 2.1662934573028525),
 ('facetime', 2.4351387695954325),
 ('seamlessly', 3.2802368096096894),
 ('airport', 3.5812668052736707),
 ('board', 2.882296800937652)]

>TF-IDF

In [127]:
def Get_TF_IDF(RDD,NroDoc):
  #Obtener el RDD con el iddocumento y la frecuencia de cada token
  TFRDD=Get_TF(RDD)
  #Obtenemos las puntuaciones IDF
  IDFRDD=Get_IDF(RDD,NroDoc)
  #Combinar para asignar a cada token un id de documento, frecuencia y puntuación IDF
  Tf_IDFRDD=TFRDD.join(IDFRDD)
  #Multiplicamos los valores de TF e IDF
  Tf_IDFRDD=Tf_IDFRDD.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
  #Reorganizamos el RDD para tener mejor comprension
  Tf_IDFRDD=Tf_IDFRDD.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
  return Tf_IDFRDD

In [129]:
NroDoc=len(Data)
BoW=Get_bow(Data)
TF_IDF=Get_TF_IDF(BoW,NroDoc)
TF_IDF.take(30)

[(1, 'store', 1, 1.105595616949241, 1.105595616949241),
 (1, 'diein', 1, 1.399423217328898, 1.399423217328898),
 (1, 'retweet', 1, 1.399423217328898, 1.399423217328898),
 (1, 'agree', 1, 1.3690792008697128, 1.3690792008697128),
 (1, 'stage', 1, 1.3798696809532192, 1.3798696809532192),
 (1, 'nyc', 1, 1.353380100659997, 1.353380100659997),
 (1, 'protester', 1, 1.388142206919209, 1.388142206919209),
 (1, 'protest', 1, 1.377146822617746, 1.377146822617746),
 (1, 'anger', 1, 1.388142206919209, 1.388142206919209),
 (1, 'misplace', 1, 1.390935107103379, 1.390935107103379),
 (2, 'range', 1, 2.2195389692560776, 2.2195389692560776),
 (2, 'trading', 1, 1.928054291498327, 1.928054291498327),
 (2, 'stock', 1, 1.457415164306585, 1.457415164306585),
 (2, 'mkt', 1, 1.968482948553935, 1.968482948553935),
 (2, 'update', 1, 1.3937460844372076, 1.3937460844372076),
 (2, 'apple', 1, 0.578668824553762, 0.578668824553762),
 (2, 'intraday', 1, 1.6270242958343457, 1.6270242958343457),
 (2, 'premium', 1, 2.0761